# Black Karasinski Model

In financial mathematics, the Black–Karasinski model is a mathematical model of the term structure of interest rates; see short-rate model. It is a one-factor model as it describes interest rate movements as driven by a single source of randomness. It belongs to the class of no-arbitrage models, i.e. it can fit today's zero-coupon bond prices, and in its most general form, today's prices for a set of caps, floors or European swaptions. The model was introduced by Fischer Black and Piotr Karasinski in 1991.


## Hull white Model Variations

$d ln(r_t) = \big(\theta(t) - a\cdot ln(r(t))\big) dt + \sigma_t dW_t$


where 
- $a$ is the mean reversion constant, 
- $\sigma$ is the volatility parameter. 
- The parameter $\theta(t)$ is chosen in order to fit the input term structure of interest rates.

What is the **"right" value for parameters $a$ and $\sigma$**? This is the question that we address by calibrating to market instruments.

The model implies a log-normal distribution for the short rate and therefore the expected value of the money-market account is infinite for any maturity.

In the **original article by Fischer Black and Piotr Karasinski the model was implemented using a binomial tree with variable spacing, but a trinomial tree implementation is more common in practice**, typically a log-normal application of the Hull–White lattice.


In [11]:
import QuantLib as ql
from collections import namedtuple
import math
import calibration_helper

In [12]:
today = ql.Date(15, ql.February, 2002);
settlement= ql.Date(19, ql.February, 2002);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [13]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [14]:
model = ql.BlackKarasinski(term_structure);
engine = ql.TreeSwaptionEngine(model, 100)
swaptions = calibration_helper.create_swaption_helpers(data, index, term_structure, engine)

optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.03949, sigma = 0.11678


In [15]:
calibration_helper.calibration_report(swaptions, data)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.00870         0.00949         0.10531         0.11480        -0.08262
        0.00967         0.01008         0.10629         0.11080        -0.04062
        0.00867         0.00872         0.10643         0.10700        -0.00533
        0.00650         0.00623         0.10664         0.10210         0.04426
        0.00355         0.00332         0.10701         0.10000         0.06978
----------------------------------------------------------------------------------
Cumulative Error :         0.12415
